# Practica 1

## Parte B

### Ejercicio 1:

Instalamos el dataset "ml-100k" (ya esta incluido con la libreria scikit-surprise).

In [ ]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505178 sha256=85bdd90b3f277c6bd532850de8f098508d4de54d2eff734fed58a615da4d3a16
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [ ]:
import os
import pandas as pd
import numpy as np

# Variable SEED, para que cada vez que se ejecute el código salgan los mismos resultados
SEED = 123

In [ ]:
# Cargar movielens-100k dataset
from surprise import Dataset

data = Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


### Ejercicio 2:

surprise tiene funcion **train_test_split** que acepta datos de entrada y se puede especificar que fraccion de datos vamos a usar para la evaluacion.

In [ ]:
# Dividir data en particiones de entrenamiento (75%) y evaluacion (25%)
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=SEED)

### Ejercicio 3/4:

En primer caso, utilizamos KNNBasic con la metrica de similtud pearson para realizar filtrado colaborativo basado en vecinos. KNN es un clasificador de aprendizaje supervisado no paramétrico que utiliza la proximidad para realizar predicciones sobre la agrupación de un punto de datos individual.

Primer caso es basado en usarios. Tambien, podemos ver detalles estadisticos como RMSE, MSE, MAE y tiempos de ejecucion de datos de entrenamiento.

In [ ]:
# Filtrado KNNBasic basado en usarios
from surprise import KNNBasic
from surprise.model_selection.validation import cross_validate

knn_basic_user = KNNBasic(sim_options={"name": "pearson", "user_based": True}, random_state=SEED)
knn_basic_user.fit(trainset)

# Mostrar los resultados
cross_validate(knn_basic_user, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0133  1.0053  1.0171  1.0100  1.0137  1.0119  0.0040  
MSE (testset)     1.0267  1.0105  1.0345  1.0200  1.0276  1.0239  0.0081  
MAE (testset)     0.8039  0.7961  0.8085  0.8010  0.8042  0.8027  0.0041  
Fit time          1.30    0.62    1.02    0.61    0.62    0.84    0.28    
Test time         3.13    2.85    3.23    2.87    6.03    3.62    1.21    


{'test_rmse': array([1.01326942, 1.00525329, 1.01708668, 1.00996748, 1.01370522]),
 'test_mse': array([1.02671492, 1.01053417, 1.03446532, 1.02003432, 1.02759828]),
 'test_mae': array([0.80386545, 0.79608709, 0.80847087, 0.80096623, 0.8041846 ]),
 'fit_time': (1.2986500263214111,
  0.618494987487793,
  1.0242464542388916,
  0.6129755973815918,
  0.6246311664581299),
 'test_time': (3.129380941390991,
  2.846304178237915,
  3.2254109382629395,
  2.870974540710449,
  6.033080101013184)}

En segundo caso, todavia utilizamos KNNBasic sino en este caso para filtrado basado en productos.

In [ ]:
# Filtrado KNNBasic basado en productos
from surprise import KNNBasic

knn_basic_product = KNNBasic(sim_options={"name": "pearson", "user_based": False}, random_state=SEED)
knn_basic_product.fit(trainset)

# Mostrar los resultados
cross_validate(knn_basic_product, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0354  1.0436  1.0462  1.0415  1.0352  1.0404  0.0044  
MSE (testset)     1.0721  1.0890  1.0945  1.0847  1.0716  1.0824  0.0091  
MAE (testset)     0.8299  0.8376  0.8382  0.8346  0.8289  0.8338  0.0038  
Fit time          0.99    1.49    1.01    0.91    0.90    1.06    0.22    
Test time         3.10    3.61    3.20    4.20    3.28    3.48    0.40    


{'test_rmse': array([1.03543189, 1.04356581, 1.04620686, 1.04150264, 1.03520073]),
 'test_mse': array([1.07211919, 1.08902961, 1.09454879, 1.08472775, 1.07164056]),
 'test_mae': array([0.82991946, 0.8375962 , 0.83824325, 0.83455163, 0.82892574]),
 'fit_time': (0.9866795539855957,
  1.4895381927490234,
  1.0133533477783203,
  0.9146215915679932,
  0.9035992622375488),
 'test_time': (3.1027908325195312,
  3.614504337310791,
  3.1985890865325928,
  4.196393728256226,
  3.284980297088623)}

En la otra parte, utilizamos SVD y NMF para realizar filtrado colaborativo basado en modelos usando factorización de matrices.

En primer caso, utilizamos SVD. La idea de este algoritmo es hacer operaciones de algebra lineal - factorización de una matriz real o compleja en una rotación, seguida de un reescalado seguido de otra rotación.

In [ ]:
# Filtrado colaborativo SVD
from surprise import SVD

svd = SVD(random_state=SEED)
svd.fit(trainset)

# Mostrar los resultados
cross_validate(svd, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9307  0.9398  0.9273  0.9420  0.9370  0.9354  0.0055  
MSE (testset)     0.8661  0.8833  0.8599  0.8873  0.8780  0.8749  0.0104  
MAE (testset)     0.7344  0.7406  0.7324  0.7392  0.7394  0.7372  0.0032  
Fit time          1.39    1.30    1.35    1.44    1.33    1.36    0.05    
Test time         0.10    0.12    0.39    0.10    0.22    0.19    0.11    


{'test_rmse': array([0.93065023, 0.93984428, 0.92728643, 0.94197911, 0.9370042 ]),
 'test_mse': array([0.86610986, 0.88330728, 0.85986012, 0.88732464, 0.87797688]),
 'test_mae': array([0.73440012, 0.74061803, 0.73240902, 0.73915711, 0.73944478]),
 'fit_time': (1.3922641277313232,
  1.2975091934204102,
  1.3522517681121826,
  1.4401121139526367,
  1.3292179107666016),
 'test_time': (0.10227560997009277,
  0.11933112144470215,
  0.39071059226989746,
  0.10284161567687988,
  0.22319602966308594)}

En segundo caso, utilizamos NMF. La factorización de matrices no negativas es un algoritmo en el que una matriz se factoriza en otras dos matrices con la propiedad de que las tres matrices no tienen elementos negativos. Esta no negatividad hace que las matrices resultantes sean más fáciles de verificar.

In [ ]:
# Filtrado colaborativo NMF
from surprise import NMF

nmf = NMF(random_state=SEED)
nmf.fit(trainset)

# Mostrar los resultados
cross_validate(nmf, data, measures=["RMSE", "MSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9589  0.9543  0.9621  0.9658  0.9621  0.9606  0.0039  
MSE (testset)     0.9195  0.9106  0.9256  0.9328  0.9256  0.9228  0.0074  
MAE (testset)     0.7564  0.7496  0.7546  0.7597  0.7563  0.7553  0.0033  
Fit time          1.95    2.01    1.98    2.76    1.98    2.14    0.31    
Test time         0.10    0.22    0.08    0.09    0.26    0.15    0.08    


{'test_rmse': array([0.95890697, 0.95427665, 0.96208591, 0.96581586, 0.96208607]),
 'test_mse': array([0.91950258, 0.91064392, 0.92560929, 0.93280028, 0.9256096 ]),
 'test_mae': array([0.75635132, 0.74960846, 0.75461102, 0.75966807, 0.75632077]),
 'fit_time': (1.9531447887420654,
  2.01442813873291,
  1.9788787364959717,
  2.756999969482422,
  1.9837486743927002),
 'test_time': (0.09654498100280762,
  0.2221992015838623,
  0.08441424369812012,
  0.08793854713439941,
  0.261502742767334)}

### Ejercicio 5:

Cuando tenemos todos algoritmos entrenados, utlizamos funcion **test** con cada de los algoritmos y le pasamos datos de evaluacion. Pero antes de eso, es importante asegurar que datos de testset son limpios y utiles. En nuestro caso, significa eliminar peliculas con rating menor de 4 porque lo interpretamos que eran peliculas malas para usario concreto.  

In [ ]:
# Hacer predicciones para conjunto de evaluacion con cada algoritmo
from surprise import accuracy

# solo tomar peliculas de testset con rating 4 o mas
testset = [x for x in testset if x[2] >= 4]

predictions_user = knn_basic_user.test(testset)
predictions_product = knn_basic_product.test(testset)
predictions_svd = svd.test(testset)
predictions_nmf = nmf.test(testset)

# Mostrar 5 resultados de cada predicciones
N = 5

res_user = predictions_user[:N]
res_product = predictions_product[:N]
res_svd = predictions_svd[:N]
res_nmf = predictions_nmf[:N]

from pprint import pprint

print("FILTRADO BASADO EN USUARIOS")
pprint(res_user)
print()
print("FILTRADO BASADO EN PRODUCTOS")
pprint(res_product)
print()
print("FILTRADO COLLABORATIVO SVD")
pprint(res_svd)
print()
print("FILTRADO COLLABORATIVO NMF")
pprint(res_nmf)
print()

# # Sort las predicciones de mejores a peores
# predictions_user.sort(key=lambda x: x.est, reverse=True)
# predictions_product.sort(key=lambda x: x.est, reverse=True)
# predictions_svd.sort(key=lambda x: x.est, reverse=True)
# predictions_nmf.sort(key=lambda x: x.est, reverse=True)

FILTRADO BASADO EN USUARIOS
[Prediction(uid='296', iid='462', r_ui=4.0, est=3.708019495021707, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='272', iid='187', r_ui=5.0, est=4.36649823387495, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='883', iid='185', r_ui=5.0, est=4.570412703163265, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='524', iid='79', r_ui=4.0, est=3.9531223102396273, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='7', iid='93', r_ui=5.0, est=3.905781836217559, details={'actual_k': 40, 'was_impossible': False})]

FILTRADO BASADO EN PRODUCTOS
[Prediction(uid='296', iid='462', r_ui=4.0, est=4.111159191448558, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='272', iid='187', r_ui=5.0, est=4.578672282025403, details={'actual_k': 35, 'was_impossible': False}),
 Prediction(uid='883', iid='185', r_ui=5.0, est=4.136015790388917, details={'actual_k': 40, 'was_impossible': Fal

En cada de las predicciones, podemos ver atributos "est" y "r_ui" que nos interesan mas. Est nos muestra la valor prediccionada (estimada) comparando con la referencia (r_ui). Estos datos nos dan la primera impresion de la calidad de predicciones de cada algoritmo.

### Ejercicio 6:

En esta parte obtenemos difetentes metrics estadisticos para evaluar cada modelo y hacer conclusiones. Empezamos por RMSE, usando la funcion **accuracy** de surprise.

In [ ]:
from surprise import accuracy

# Evaluacion RMSE
rmse_user = accuracy.rmse(predictions_user, verbose=True)
rmse_product = accuracy.rmse(predictions_product, verbose=True)
rmse_svd = accuracy.rmse(predictions_svd, verbose=True)
rmse_nmf = accuracy.rmse(predictions_nmf, verbose=True)

RMSE: 0.7138
RMSE: 0.7851
RMSE: 0.6583
RMSE: 0.7398


! IMPORTANTE: he hecho la version de enunciado antigua (implementado precision@k y recall@k) !

La funcion siguiente hace numerosas operaciones para obtener precision@k:
1. agrupa todas las predicciones por usario;
2. para cada usario, ordena los items por el rating prediccionado;
3. marca los primeros k items como recomendados (1) y el resto como no recomendados (0);
4. crea un vector de valores verdaderos en el que cada item se considera relevante si su rating real es mayor a 4;
5. calcula la precision usando precision_score de sklrean para cada usuario.

Precision es util porque nos muestra la fraccion de instancias relevantes entre las instancias recuperadas.

In [ ]:
K = 10

# Evaluacion precision usando precision_score
import numpy as np
from sklearn.metrics import precision_score
from collections import defaultdict

def precision_at_k(predictions, k=10, threshold=4.0):
    # grupa predicciones por usario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, true_r, est))

    precisions = []
    for uid, preds in user_preds.items():
        # ordena predicciones
        preds_sorted = sorted(preds, key=lambda x: x[2], reverse=True)
        n = len(preds_sorted)

        # crea vector binario de prediccion (relevante = 1, irrelevante = 0).
        y_pred = np.zeros(n)
        top_k = min(k, n)
        y_pred[:top_k] = 1

        y_true = np.array([1 if true_r > threshold else 0 for (_, true_r, _) in preds_sorted])

        # calcula precision
        prec = precision_score(y_true, y_pred, zero_division=0)
        precisions.append(prec)

    return np.mean(precisions)

In [ ]:
# obtenemos los valores de precision
precision_user = precision_at_k(predictions_user, k=K)
precision_product = precision_at_k(predictions_product, k=K)
precision_svd = precision_at_k(predictions_svd, k=K)
precision_nmf = precision_at_k(predictions_nmf, k=K)

print("Precision@10 (basado en usario):", precision_user)
print("Precision@10 (basado en productos):", precision_product)
print("Precision@10 (SVD):", precision_svd)
print("Precision@10 (NMF):", precision_nmf)

Precision@10 (basado en usario): 0.45457496346147314
Precision@10 (basado en productos): 0.44526018830087355
Precision@10 (SVD): 0.45114883926447097
Precision@10 (NMF): 0.42341864654498484


En la funcion **recall_at_k** la preparacion es parecida a la de **precision_at_k**, sino al final utilizamos recall_score para obtener recall. La metrica de recall muestra la fraccion de instancias relevantes que se recuperaron.

In [ ]:
import numpy as np
from collections import defaultdict
from sklearn.metrics import recall_score

# Evaluacion recall usando recall_score
def recall_at_k(predictions, k=10, threshold=4.0):
    # grupa predicciones por usario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, true_r, est))

    recalls = []
    for uid, preds in user_preds.items():
        # ordena predicciones
        preds_sorted = sorted(preds, key=lambda x: x[2], reverse=True)
        n = len(preds_sorted)

        # crea vector binario de prediccion (relevant = 1, irrelevante = 0).
        y_pred = np.zeros(n)
        top_k = min(k, n)
        y_pred[:top_k] = 1

        y_true = np.array([1 if true_r > threshold else 0 for (_, true_r, _) in preds_sorted])

        # calcula recall
        rec = recall_score(y_true, y_pred, zero_division=0)
        recalls.append(rec)

    return np.mean(recalls)

In [ ]:
# obtenemos los valores de recall
recall_user = recall_at_k(predictions_user, k=K)
recall_product = recall_at_k(predictions_product, k=K)
recall_svd = recall_at_k(predictions_svd, k=K)
recall_nmf = recall_at_k(predictions_nmf, k=K)

print("Recall@10 (basado en usario):", recall_user)
print("Recall@10 (basado en productos):", recall_product)
print("Recall@10 (SVD):", recall_svd)
print("Recall@10 (NMF):", recall_nmf)

Recall@10 (basado en usario): 0.7298280166808881
Recall@10 (basado en productos): 0.7199675225974883
Recall@10 (SVD): 0.7253251678567316
Recall@10 (NMF): 0.6978109286773364


La funcion de NDCG hace las siguientes operaciones:
1. agrupa todas las predicciones por usario;
2. ordena las predicciones por rating estimado descendiente;
3. crea un vector binario de relevancia (1 si el rating real es mayor que el 4, 0 si no) para los top k items;
4. crea un vector con los ratings estimados correspondientes a esos items;
5. calcula el NDCG para cada usuario usando estos vectores.

La metrica NDCG muestra la relevancia de los elementos devueltos por el modelo comparado con la relevancia del elemento que devolvería un modelo “ideal” hipotético.

In [ ]:
import numpy as np
from collections import defaultdict
from sklearn.metrics import ndcg_score

# Evaluacion recall usando ndcg_score
def ndcg_at_k(predictions, k=10, threshold=4.0):
    # grupa predicciones por usario
    user_preds = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_preds[uid].append((iid, true_r, est))

    ndcg_list = []
    for uid, items in user_preds.items():
        # ordena predicciones
        items_sorted = sorted(items, key=lambda x: x[2], reverse=True)
        top_items = items_sorted[:k]
        n_items = len(top_items)

        # crea vector binario de prediccion y vector de valores estimados
        y_true = np.array([[1 if true_r > threshold else 0 for (_, true_r, _) in top_items]])
        y_scores = np.array([[est for (_, _, est) in top_items]])

        # si hay solo un elemento, agrega un elemento fake (0 relevancia y 0 puntuación)
        if n_items < 2:
            pad = 2 - n_items
            y_true = np.concatenate([y_true, np.zeros((1, pad))], axis=1)
            y_scores = np.concatenate([y_scores, np.zeros((1, pad))], axis=1)
            current_k = 2
        else:
            current_k = n_items

        # calcula NDCG
        ndcg_value = ndcg_score(y_true, y_scores, k=current_k)
        ndcg_list.append(ndcg_value)

    return np.mean(ndcg_list)

In [ ]:
ndcg_user = ndcg_at_k(predictions_user, k=K)
ndcg_product = ndcg_at_k(predictions_product, k=K)
ndcg_svd = ndcg_at_k(predictions_svd, k=K)
ndcg_nmf = ndcg_at_k(predictions_nmf, k=K)

print("NDCG@10 (basado en usario):", ndcg_user)
print("NDCG@10 (basado en productos):", ndcg_product)
print("NDCG@10 (SVD):", ndcg_svd)
print("NDCG@10 (NMF):", ndcg_nmf)

NDCG@10 (basado en usario): 0.7894041307054076
NDCG@10 (basado en productos): 0.7664042756170181
NDCG@10 (SVD): 0.766710880762066
NDCG@10 (NMF): 0.7248258623159879


In [ ]:
# crear tabla consolidada de RMSE, precision, recall, ndcg
import pandas as pd

data = {'Algoritmo': ['Basado en Usuarios', 'Basado en Productos', 'SVD', 'NMF'],
  'RMSE': [
    rmse_user,
    rmse_product,
    rmse_svd,
    rmse_nmf
  ],
  'Precision': [
    precision_user,
    precision_product,
    precision_svd,
    precision_nmf
  ],
  'Recall': [
    recall_user,
    recall_product,
    recall_svd,
    recall_nmf
  ],
  'NDCG': [
    ndcg_user,
    ndcg_product,
    ndcg_svd,
    ndcg_nmf
  ]
}

df = pd.DataFrame(data)
df

,Algoritmo,RMSE,Precision,Recall,NDCG
0,Basado en Usuarios,0.713799,0.454575,0.729828,0.789404
1,Basado en Productos,0.785114,0.445260,0.719968,0.766404
2,SVD,0.658330,0.451149,0.725325,0.766711
3,NMF,0.739817,0.423419,0.697811,0.724826


Análisis de los resultados:

Basado en Usuarios
- En general, sobresale en las metricas de recomendacion (precision, recall y NDCG), aunque no es el mejor en RMSE.

Basado en Productos
- En general, funciona algo peor que el basado en usuarios en todas las metricas.

SVD
- Es buena para predecir la calificacion numerica (menor RMSE), pero su ranking (NDCG) no es el mas alto.

NMF
- En general, es el metodo que rinde peor en estas metricas comparado con los otros tres.

Conclusiones:

Si la prioridad es la exactitud numerica de las predicciones (que el sistema acierte con la calificacion lo mas cerca posible a la real), el mejor es SVD porque tiene el RMSE mas bajo.

Si la prioridad es la calidad de la recomendacion en terminos de relevancia (cantidad de items que realmente le gustan al usario), en este caso KNNBasic basado en Usuarios es superior (mejor precisión, recall y NDCG).

Dado que en la mayoría de escenarios de recomendacion se valoran mas las metricas de relevancia (precision, recall, NDCG) que la exactitud absoluta de, el metodo basado en usuarios puede ser la mejor eleccion global entre los algoritmos presentados.